SOLID Principles, Mixins
- Design Patterns: Singleton, Factory, Strategy, Observer
- Hands-On Lab: Build modular plugin architecture
- Bonus: Authentication flow using OAuth and Azure AD

Advanced Python Concepts
- Decorators: Basics, Chaining, Real Use Cases
- Context Managers: Built-in and Custom using __enter__/__exit__
- Generators & Coroutines: yield, send, generator pipelines
- Metaclasses: Basics and Framework Applications
- Hands-On Lab: Create a logger using decorators and context managers

Decorator Example:
```python

It is AOP = Aspect-Oriented Programming, its similar to Attributes in C# or Annotations in Java

Function takes other function as an argument

In [9]:
#loggin decorator in python
def logging_decorator(func):
    def wrapper(*args, **kwargs):
        print(f"Calling function '{func.__name__}' with arguments: {args} and keyword arguments: {kwargs}")
        result = func(*args, **kwargs)
        print(f"Function '{func.__name__}' returned: {result}")
        return result
    return wrapper

@logging_decorator
def add(a, b, **kwargs):
    print(f"Addition called")
    return a + b

@logging_decorator
def helloWord():
    print(f"Hw called")
    return "Hello World!"


helloWord()


add(3, 5, extra_param="example", another_param=42)

Calling function 'helloWord' with arguments: () and keyword arguments: {}
Hw called
Function 'helloWord' returned: Hello World!
Calling function 'add' with arguments: (3, 5) and keyword arguments: {'extra_param': 'example', 'another_param': 42}
Addition called
Function 'add' returned: 8


8

Validation Decorator Example:
```python

In [ ]:
#Validation decorator in python
def validation_decorator(func):
    def wrapper(*args, **kwargs):
        #Tuple comprehension to validate arguments of function
        if any(not isinstance(x, (int or float)) for x in args):
            raise ValueError("All arguments must be numbers.")
        
        if any(x == 0 for x in args if isinstance(x, (int or float))):
            raise ValueError("All arguments must be numbers greater than zero.")
        return func(*args, **kwargs)
    return wrapper

@logging_decorator
@validation_decorator
def multiply(a, b):
    print(f"Multiplication called  inside multiply function")
    val= a * b
    print(f"Multiplication result: {val}")
    return val

multiply(3, 5)  # This will work

# try:
#     multiply("", 5)  # This will raise a ValueError
# except ValueError as e:
#     print(f"Error: {e}")

Calling function 'wrapper' with arguments: (3, 5) and keyword arguments: {}
Multiplication called
Multiplication result: 15
Function 'wrapper' returned: 15


15

Context Manager Python  = using statement in C#
```python

You can create your own context manager using __enter__ and __exit__ methods


In [ ]:
#file handling creating and adding line of text 
with open("abc.txt","a") as file:
    file.write('First line in the file')

with open("abc.txt","r") as file:
    lines = file.read()
    print(f"File content: {lines}")

print(f"written file")

File content: First line in the file
written file


In [29]:
%%writefile my_context_manager.py
#create my own custom context manager 
class MyContextManager:
    def __enter__(self):
        print("Entering the context")
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        print("Exiting the context")
        if exc_type is not None:
            print(f"An exception occurred: {exc_value}")

    def do_something(self):
        print("Doing something inside the context manager")


Writing my_context_manager.py


In [ ]:
from my_context_manager import MyContextManager

with MyContextManager() as manager:
    manager.do_something()
    # Uncomment the next line to see exception handling in action
    # raise ValueError("An error occurred inside the context manager")

Entering the context
Doing something inside the context manager
Exiting the context


In [33]:
from contextlib import contextmanager   

@contextmanager
def my_context_manager():
    print("Entering the context")
    try:
        yield
    except Exception as e:
        print(f"An exception occurred: {e}")
    finally:
        print("Exiting the context")

with my_context_manager() as cm:
    cm.do_something()

Entering the context
An exception occurred: 'NoneType' object has no attribute 'do_something'
Exiting the context


Generator in python created using yield keyword, same is there in C#

In [ ]:
#simple generator function
def my_generator():
    print("Generator started")
    yield 1
    print("Generator resumed")
    yield 2
    print("Generator finished")

# calling the generator manually    
g=my_generator() 

print(g.__next__())  # Output: 1
print(g.__next__())  # Output: 2
try:
    print(g.__next__())  # Raises StopIteration exception
except StopIteration:
    print("Generator has no more items")
finally:
    print("Generator has been exhausted")   


# Using the generator in a loop
for value in my_generator():
    print(f"Yielded value: {value}")
    

Generator started
1
Generator resumed
2
Generator finished
Generator has no more items
Generator has been exhausted
Generator started
Yielded value: 1
Generator resumed
Yielded value: 2
Generator finished


Meta classes in Python are classes of classes, they define how classes behave
```python

It help to create APIs, ORM, and other frameworks

It help to create custom class creation logic


It help to enforce rules on class creation, like validation or modification of attributes

In [46]:
#Define metaclass
class MyMeta(type):
    def __new__(cls, name, bases, attrs):
        print(f"Creating class {name} with MyMeta")
        return super().__new__(cls, name, bases, attrs)
    
class MyClass(metaclass=MyMeta):
    def __init__(self, value):
        self.value = value

    def display(self):
        print(f"Value: {self.value}")

obj = MyClass(42)
obj.display()  # Output: Value: 42

Creating class MyClass with MyMeta
Value: 42


In [1]:
class BankMeta(type):
    def __new__(cls, name, bases, attrs):
        print(f"Creating class {name} with BankMeta")

        newclass = super().__new__(cls, name, bases, attrs)

        if not hasattr(newclass, 'deposit') or not hasattr(newclass, 'withdraw'):
            raise ValueError("Class must have a deposit and withdraw method")
        
        def log_transaction(self):
            print(f"Transaction logged for {newclass.__name__}")

        def amount_validator(func):
            def wrapper(self, amount):
                if amount <= 0:
                    raise ValueError("Amount must be greater than zero.")
                return func(self, amount)
            return wrapper

        # Adding the log_transaction method to the new class
        setattr(newclass, 'log_transaction', log_transaction)

        # Wrap deposit and withdraw with amount_validator
        setattr(newclass, 'deposit', amount_validator(getattr(newclass, 'deposit')))
        setattr(newclass, 'withdraw', amount_validator(getattr(newclass, 'withdraw')))


        # Adding a class variable to the new class
        setattr(newclass, 'table_name', "My Bank")

        return newclass



class BankAccount(metaclass=BankMeta):
    def __init__(self, account_number, balance):
        self.account_number = account_number
        self.balance = balance
    
    def deposit(self, amount):
        self.balance += amount
        self.log_transaction()
        print(f"Deposited {amount}. New balance: {self.balance}")

    def withdraw(self, amount): 
        if amount > self.balance:
            raise ValueError("Insufficient funds for withdrawal.")
        self.balance -= amount
        self.log_transaction()
        print(f"Withdrew {amount}. New balance: {self.balance}")    

    def display(self):
        print(f"Account Number: {self.account_number}, Balance: {self.balance}, Table Name: {self.table_name}")


obj = BankAccount("123456", 1000)
obj.display()  # Output: Account Number: 123456, Balance: 1000, Table Name: My Bank
obj.deposit(500)  # Output: Deposited 500. New balance: 1500
obj.withdraw(20)  

Creating class BankAccount with BankMeta
Account Number: 123456, Balance: 1000, Table Name: My Bank
Transaction logged for BankAccount
Deposited 500. New balance: 1500
Transaction logged for BankAccount
Withdrew 20. New balance: 1480
